# AgentOps Langchain Agent Implementation

Using AgentOps monitoring with Langchain is simple. We've created a LangchainCallbackHandler that will do all of the heavy lifting!

First we'll import the typical Langchain packages:

In [1]:
import agentops

# %pip install langchain==0.2.9

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import tool, AgentExecutor, create_openai_tools_agent

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [3]:
from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialized, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, default_tags=["Langchain Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

# You must pass in a callback handler to record your agent
llm.callbacks = [agentops_handler]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
        # ("tool_names", "find_movie")
    ]
)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=418f60fc-a709-4a61-952c-6b9670406198


You can also retrieve the `session_id` of the newly created session.

In [6]:
print("Agent Ops session ID: " + str(agentops_handler.current_session_ids))

Agent Ops session ID: ['418f60fc-a709-4a61-952c-6b9670406198']


Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [7]:
@tool
def find_movie(genre: str) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [8]:
for t in tools:
    t.callbacks = [agentops_handler]

Add the tools to our LLM

In [9]:
llm_with_tools = llm.bind_tools([find_movie])

Finally, let's create our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [10]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
agent_executor.invoke(
    {"input": "What comedies are playing?"}, config={"callback": [agentops_handler]}
)

{'input': 'What comedies are playing?',
 'output': '"Pineapple Express" está disponible para ver. ¿Te gustaría saber más sobre esta película?'}

Now if we look in the AgentOps dashboard, you will see a session recorded with the LLM calls and tool usage.

In [ ]:
assert False

## Langchain V0.1 Example
This example is out of date.

In [ ]:
%pip install langchain==0.1.6

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [ ]:
from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
from dotenv import load_dotenv

load_dotenv()

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialized, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard.

In [ ]:
AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, default_tags=["Langchain Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

You can also retrieve the `session_id` of the newly created session.

In [ ]:
print("Agent Ops session ID: " + str(agentops_handler.current_session_ids))

Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [ ]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [ ]:
for t in tools:
    t.callbacks = [agentops_handler]

Finally, let's use our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    callbacks=[
        agentops_handler
    ],  # You must pass in a callback handler to record your agent
    handle_parsing_errors=True,
)

In [ ]:
agent.invoke("What comedies are playing?", callbacks=[agentops_handler])

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)
![image.png](attachment:3d9393fa-3d6a-4193-b6c9-43413dc19d15.png)

# Async Agents

Several langchain agents require async callback handlers. AgentOps also supports this.

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

In [ ]:
from agentops.partners.langchain_callback_handler import (
    AsyncLangchainCallbackHandler as AgentOpsAsyncLangchainCallbackHandler,
)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
agentops_handler = AgentOpsAsyncLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Async Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

print("Agent Ops session ID: " + str(await agentops_handler.session_id))

In [ ]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

for t in tools:
    t.callbacks = [agentops_handler]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    callbacks=[agentops_handler],
)

await agent.arun("What comedies are playing?")

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)

![image.png](attachment:69f2121a-d437-4c09-bbbe-c76c9243ee19.png)